In [1]:
# Install OpenAI library
!pip install -U -q openai tenacity

In [2]:
import pandas as pd
from openai import OpenAI
import json,os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
filepath = '/content/drive/MyDrive/chatbot_llm/'

# **Initialize the OpenAI**

In [5]:
# Read the OpenAI API key
import openai

with open(filepath + "OpenAI_API_Key.txt", "r") as f:
  openai.api_key = ' '.join(f.readlines())
openai_client = OpenAI(api_key=openai.api_key)
  #os.environ['OPENAI_API_KEY'] = openai.api_key

# Read the data

In [6]:
data = pd.read_json(filepath + "intents (1).json")

In [7]:
data.head(50)

,intents
0,"{'tag': 'greeting', 'patterns': ['Hi', 'Hey', ..."
1,"{'tag': 'morning', 'patterns': ['Good morning'..."
2,"{'tag': 'afternoon', 'patterns': ['Good aftern..."
3,"{'tag': 'evening', 'patterns': ['Good evening'..."
4,"{'tag': 'night', 'patterns': ['Good night'], '..."
5,"{'tag': 'goodbye', 'patterns': ['Bye', 'See yo..."
6,"{'tag': 'thanks', 'patterns': ['Thanks', 'Than..."
7,"{'tag': 'no-response', 'patterns': [''], 'resp..."
8,"{'tag': 'neutral-response', 'patterns': ['noth..."
9,"{'tag': 'about', 'patterns': ['Who are you?', ..."


In [8]:
data = data['intents']

In [9]:
len(data)

80

In [10]:
train_data = pd.DataFrame(columns=['intents','patterns','responses'])

## Making data a bit readable

In [11]:
for index in data:
        new_data = {'intents': index['tag'].lower(),'patterns': index['patterns'],'responses':index['responses']}
        train_data = train_data._append(new_data, ignore_index=True)

## Initializing the conversation

In [12]:
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"

    mental_health_issues_category = ['greeting','morning','afternoon','evening','night','goodbye','thanks',
                                     'no-response','neutral-response','about','skill','creation','name','help',
                                     'sad','stressed','worthless','depressed','happy','casual','anxious',
                                     'not-talking','sleep','scared','death','understand','done','suicide',
                                     'hate-me','default','jokes','repeat','wrong','stupid','location','something-else',
                                     'friends','ask','problem','no-approach','learn-more','user-agree','meditation',
                                     'user-meditation','pandora-useful','user-advice','learn-mental-health',
                                     'mental-health-fact','fact-1','fact-2','fact-3','fact-5','fact-6','fact-7',
                                     'fact-8','fact-9','fact-10','fact-11','fact-12','fact-13','fact-14','fact-15',
                                     'fact-16','fact-17','fact-18','fact-19','fact-20','fact-21','fact-22','fact-23',
                                     'fact-24','fact-25','fact-26','fact-27','fact-28','fact-29','fact-30','fact-31','fact-32']

    response_format = {'issue': '_'}

    system_message = f"""
    You are an intelligent mental healthcare expert and your goal is to find the health issue of the patient.
    You need to ask relevant questions and understand the users issue by analysing the user's responses.
    You final objective is to find the health issue and categorise them accordingly based on the {mental_health_issues_category} values. The values must be based on these category or you will be heavily penalised.
    The final response should in the {response_format} format with JSON value filled in the issue key. The response must be in JSON format with no additional text.

    To understand the users request and problems you need to have the following chain of thoughts:
    Follow the chain-of-thoughts below and only output the final mental health issue. \n
    {delimiter}
    Thought 1: Ask a question to understand the user's health issues with his mentality. \n
    If the health issue is unclear. Ask followup questions to understand their issue little deeper.
    categorise the health issues in that should present in the {mental_health_issues_category}.
    Remember the instructions around the values for the different mental problems.
    If the necessary information has been extracted, only then proceed to the next step. \n
    Otherwise, rephrase the question to capture their problems clearly. \n
    {delimiter}
    Thought 2: Ask indirect questions to understand the mental issues if user is not able to clarify clearly. \n
    Ask questions relevant to {mental_health_issues_category} and categorise the health issues which matches the issues present in {mental_health_issues_category}.
    If only the necessary information has been extracted, then only proceed to the next step. \n
    Otherwise, rephrase the question to capture their problems clearly. \n
    {delimiter}
    Here is a sample conversation between the user and assistant:
    User: "I am feeling lonely"
    Assistant: "I'm sorry to hear that. I'm here for you. Talking about it might help. So, tell me why do you think you're feeling this way?"
    User: "How can I maintain social connections? What if I feel lonely"
    Assistant: "A lot of people are alone right now, but we don't have to be lonely. We're all in this together. Think about the different ways to connect that
    are most meaningful for you. For example, you might prefer a video chat over a phone call, or you might prefer to text throughout the day rather than one
    set time for a video call. Then, work with your social networks to make a plan. You might video chat with your close friends in the evening and phone a
    family member once a week. Remember to be mindful of people who may not be online. Check in by phone and ask how you can help.  Mindlessly scrolling through
    social media and liking a few posts usually doesn't build strong social connections. Make sure you focus on strategies that actually make you feel included
    and connected. If your current strategies don't help you feel connected, problem-solve to see if you can find a solution. Everyone feels lonely at times.
    Maybe you recently moved to a new city, are changing your circle of friends, lost someone important in your life, or lost your job and also lost important
    social connections with coworkers. Other people may have physical connections to others but may feel like their emotional or social needs aren't met.
    Measures like social distancing or self-isolation can make loneliness feel worse no matter why you feel lonely now. Reach out to the connections you do have.
    Suggest ways to keep in touch and see if you can set a regular time to connect. People may hesitate to reach out for a lot of different reasons, so don't be
    afraid to be the one who asks. Look for local community support groups and mutual aid groups on social media.This pandemic is bringing everyone together,
    so look for opportunities to make new connections. These groups are a great way to share your skills and abilities or seek help and support.Support groups
     are moving online, and there are a lot of different support lines to call if you need to talk to someone."
    Assistant: "{response_format}"
    {delimiter}

    Start with a short welcome message and encourage the user to share their mental disturbences.
    """
    conversation = [{"role": "system", "content": system_message}]
    return conversation

In [13]:
debug_conversation = initialize_conversation()
print(debug_conversation)

[{'role': 'system', 'content': '\n    You are an intelligent mental healthcare expert and your goal is to find the health issue of the patient.\n    You need to ask relevant questions and understand the users issue by analysing the user\'s responses.\n    You final objective is to find the health issue and categorise them accordingly based on the [\'greeting\', \'morning\', \'afternoon\', \'evening\', \'night\', \'goodbye\', \'thanks\', \'no-response\', \'neutral-response\', \'about\', \'skill\', \'creation\', \'name\', \'help\', \'sad\', \'stressed\', \'worthless\', \'depressed\', \'happy\', \'casual\', \'anxious\', \'not-talking\', \'sleep\', \'scared\', \'death\', \'understand\', \'done\', \'suicide\', \'hate-me\', \'default\', \'jokes\', \'repeat\', \'wrong\', \'stupid\', \'location\', \'something-else\', \'friends\', \'ask\', \'problem\', \'no-approach\', \'learn-more\', \'user-agree\', \'meditation\', \'user-meditation\', \'pandora-useful\', \'user-advice\', \'learn-mental-health

In [14]:
print(debug_conversation[0]['content'])


    You are an intelligent mental healthcare expert and your goal is to find the health issue of the patient.
    You need to ask relevant questions and understand the users issue by analysing the user's responses.
    You final objective is to find the health issue and categorise them accordingly based on the ['greeting', 'morning', 'afternoon', 'evening', 'night', 'goodbye', 'thanks', 'no-response', 'neutral-response', 'about', 'skill', 'creation', 'name', 'help', 'sad', 'stressed', 'worthless', 'depressed', 'happy', 'casual', 'anxious', 'not-talking', 'sleep', 'scared', 'death', 'understand', 'done', 'suicide', 'hate-me', 'default', 'jokes', 'repeat', 'wrong', 'stupid', 'location', 'something-else', 'friends', 'ask', 'problem', 'no-approach', 'learn-more', 'user-agree', 'meditation', 'user-meditation', 'pandora-useful', 'user-advice', 'learn-mental-health', 'mental-health-fact', 'fact-1', 'fact-2', 'fact-3', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9', 'fact-10', 'fact-11', 'f

## OpenAI call for chat completions

In [15]:
# Define a Chat Completions API call
def get_chat_completions(input):
        MODEL = 'gpt-3.5-turbo'
        chat_completion = openai_client.chat.completions.create(
            model = MODEL,
            messages = input,
            seed = 2345)
        output = chat_completion.choices[0].message.content
        return output

In [16]:
input_prompt ='What is the capital of France?'
messages = [{'role':'user','content':input_prompt}]

In [17]:
get_chat_completions(messages)

'The capital of France is Paris.'

## iterate_response() - Helper Function:

In [18]:
def iterate_llm_response(funct, debug_response, num = 10):
    """
    Calls a specified function repeatedly and prints the results.
    This function is designed to test the consistency of a response from a given function.
    It calls the function multiple times (default is 10) and prints out the iteration count,
    the function's response(s).
    Args:
        funct (function): The function to be tested. This function should accept a single argument
                          and return the response value(s).
        debug_response (dict): The input argument to be passed to 'funct' on each call.
        num (int, optional): The number of times 'funct' will be called. Defaults to 10.
    Returns:
        This function only returns the results to the console.
    """
    i = 0  # Initialize counter

    while i < num:  # Loop to call the function 'num' times

        response = funct(debug_response)  # Call the function with the debug response

        # Print the iteration number, result, and reason from the response
        print("Iteration: {0}".format(i))
        print(response)
        print('-' * 50)  # Print a separator line for readability
        i += 1  # Increment the counter

# Example usage: test the consistency of responses from 'intent_confirmation_layer'
# iterate_llm_response(get_chat_completions, messages)

In [19]:
debug_user_input = "Good morning,I feel stuck."

In [20]:
debug_conversation.append({"role": "user", "content": debug_user_input})
# print(debug_conversation[0]["content"]) # System Message
print(debug_conversation[1]["content"]) # User Input

Good morning,I feel stuck.


In [21]:
display(debug_conversation)

[{'role': 'system',
  'content': '\n    You are an intelligent mental healthcare expert and your goal is to find the health issue of the patient.\n    You need to ask relevant questions and understand the users issue by analysing the user\'s responses.\n    You final objective is to find the health issue and categorise them accordingly based on the [\'greeting\', \'morning\', \'afternoon\', \'evening\', \'night\', \'goodbye\', \'thanks\', \'no-response\', \'neutral-response\', \'about\', \'skill\', \'creation\', \'name\', \'help\', \'sad\', \'stressed\', \'worthless\', \'depressed\', \'happy\', \'casual\', \'anxious\', \'not-talking\', \'sleep\', \'scared\', \'death\', \'understand\', \'done\', \'suicide\', \'hate-me\', \'default\', \'jokes\', \'repeat\', \'wrong\', \'stupid\', \'location\', \'something-else\', \'friends\', \'ask\', \'problem\', \'no-approach\', \'learn-more\', \'user-agree\', \'meditation\', \'user-meditation\', \'pandora-useful\', \'user-advice\', \'learn-mental-heal

In [22]:
# Getting the response from the Assistant by passing the conversation to the Chat Completions API
debug_response_assistant = get_chat_completions(debug_conversation)
display(debug_response_assistant)

"I'm sorry to hear that you're feeling stuck. It's important to address these feelings. Can you tell me why you think you're feeling this way?"

In [23]:
# Let's append this to the conversation list
debug_conversation.append(({"role": "system", "content": debug_response_assistant}))
debug_conversation

[{'role': 'system',
  'content': '\n    You are an intelligent mental healthcare expert and your goal is to find the health issue of the patient.\n    You need to ask relevant questions and understand the users issue by analysing the user\'s responses.\n    You final objective is to find the health issue and categorise them accordingly based on the [\'greeting\', \'morning\', \'afternoon\', \'evening\', \'night\', \'goodbye\', \'thanks\', \'no-response\', \'neutral-response\', \'about\', \'skill\', \'creation\', \'name\', \'help\', \'sad\', \'stressed\', \'worthless\', \'depressed\', \'happy\', \'casual\', \'anxious\', \'not-talking\', \'sleep\', \'scared\', \'death\', \'understand\', \'done\', \'suicide\', \'hate-me\', \'default\', \'jokes\', \'repeat\', \'wrong\', \'stupid\', \'location\', \'something-else\', \'friends\', \'ask\', \'problem\', \'no-approach\', \'learn-more\', \'user-agree\', \'meditation\', \'user-meditation\', \'pandora-useful\', \'user-advice\', \'learn-mental-heal


# checking for unneccesary messages


In [24]:
def moderation_check(user_input):
    # Call the OpenAI API to perform moderation on the user's input.
    response = openai_client.moderations.create(input=user_input)

    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    if response.results[0].flagged == True:
        # If flagged, return "Flagged"
        return "Flagged"
    else:
        # If not flagged, return "Not Flagged"
        return "Not Flagged"

In [25]:
moderation_check("I want to kill them.")

'Flagged'

In [26]:
moderation_check(debug_response_assistant)

'Not Flagged'

In [27]:
def info_mental_health(intent):
    data = None
    try:
        json_data = json.loads(intent)
        if 'issue' in json_data:
            checkIntent = json_data['issue'].lower() in train_data['intent'].values
            if(checkIntent):
                #we found the intent from chatbot, lets retrieve and show the result
                data_1 = train_data[train_data['intent'] == json_data['issue'].lower()]['response']
                data = data_1[data_1.index[0]][0]
                return data
        else:
            return data
    except ValueError as e:
        print(e)
        return data

In [28]:
def dictionary_present(response):
    delimiter = "####"

    data_format = {"issue": "user-meditation"}

    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {data_format}.
            Your task is to just extract the relevant values from the input and return only the python dictionary in JSON format with double quotes in keys and values.
            The output should match the format as {data_format}.
            {delimiter}
            The output should contain the exact keys and values as present in the input.
            Here are some sample input output pairs for better understanding:
            {delimiter}
            input 1: - Thank you for clarifying. {{'issue': 'user-meditation'}}
            output 1: {{"issue": "user-meditation"}}

            input 2: - Thank you for clarifying. Based on your symptom of mental illness. {{'issue': 'pandora-useful'}}
            output 2: {{"issue": "pandora-useful"}}
            {delimiter}
            """
    messages = [{"role": "system", "content":prompt },
                {"role": "user", "content":f"""Here is the user input: {response}""" }]

    confirmation = get_chat_completions(messages)

    return confirmation

In [29]:
string_dict = "Thank you for clarifying. Based on your questions there may be some doubts about your mental issues{'issue': 'pandora-useful'}"
check_dict = dictionary_present(string_dict)

In [30]:
check_dict

'{"issue": "pandora-useful"}'

In [33]:
def dialogue_mgmt_system():
    conversation = initialize_conversation()

    introduction = get_chat_completions(conversation)

    display(introduction + '\n')

    user_input = ''

    while(user_input != "exit"):
        user_input = input("")

        moderation = moderation_check(user_input)
        if moderation == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break

        conversation.append({"role": "user", "content": user_input})
        response_assistant = get_chat_completions(conversation)
        moderation = moderation_check(response_assistant)
        if moderation == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break
        print('Please wait while we understand the problem...\n')
        ## if json present in the text, extract it
        extract_json = dictionary_present(response_assistant)
        ## check if the issue is actually found yet
        confirmation = info_mental_health(extract_json)
        if(confirmation is None):
            ## issue not found, ask more questions to understand the intent
            print(response_assistant)
            conversation.append({"role": "assistant", "content": str(response_assistant)})
        else:
            ## print the reply
            print(str(confirmation) + '\n')

In [36]:
dialogue_mgmt_system()

"Hello, I'm here to listen and assist you with any mental health issues you may be facing. Please feel free to share what's on your mind.\n"

exit
Please wait while we understand the problem...

I'm here to help you with any mental health issues you might be facing. Feel free to share whatever is on your mind.


In [37]:
descriptions = [
    {
        "name": "information",
        "description": "information about metal illness",
        "parameters": {
            "type": "object",
            "properties": {
                "issue": {
                    "type": "string",
                    "description": "The query to be solved",
                }
            },
            "required": ["issue"],
        },
    }
]

In [38]:
def get_chat_completions_functions(input):
        MODEL = 'gpt-3.5-turbo-0613'
        chat_completion = openai_client.chat.completions.create(
            model = MODEL,
            messages = input,
            functions=descriptions,
            function_call="auto",
            seed = 2345)
        output = chat_completion.choices[0].message
        return output

In [39]:
output = get_chat_completions_functions([{"role": "user", "content": "I can't take it anymore"}])

In [40]:
output

ChatCompletionMessage(content="I'm really sorry to hear that you're feeling this way. It's important to reach out to someone who can offer support, such as a close friend, family member, or mental health professional. They can provide you with assistance and resources to help you through this difficult time.", role='assistant', function_call=None, tool_calls=None)